<a href="https://colab.research.google.com/github/zshamroukh/Colab/blob/main/api_use_case_phx_plpf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Accessing Real Time Residential Real Estate Data in 3 Steps via the Parcl Labs API

#Step 1: Register for the API

You can retrieve Parcl Labs API data with only a couple lines of GraphQL code. For demonstration purposes here we will show you a use case using Python, one of the most popular programming languages for data analysis, but the API can be used in combination with your favorite tech stack. Once you’ve received your key, by signing up [here](https://www.parcllabs.com/contact?utm_source=ParclLabs&utm_medium=Blog&utm_campaign=API-USE-CASE-BLOG) for free, you can store it as a variable in Python along with the API URL:

In [ ]:
import os
import requests
import pandas as pd
import plotly.express as px

# Storing the API credentials as an environment variable
url = "https://api-pilot.parcl.co/v1/graphql"
bearer_token = os.environ['PARCL_LABS_API_PILOT_KEY']

# Step 2: Define the GraphQL query

Let’s use the API to see how prices are evolving in real time in the Phoenix Metropolitan Area. First we name our query, in this case PHX_PLPF, and then we query the “Phoenix-Mesa-Chandler, AZ” MSA from the MSA table (see the Parcl Labs API [docs](https://docs.parcllabs.com/docs/msa) for a comprehensive list of geographies and objects available in the API).

After defining the MSA, we choose the columns we want to output - MSA_Name, PARCL_ID (our unique identifier of different levels of geography), PARCL_PRICE_FEED and DATE. Before calling the price feed, we need to define the parameters of our query. We only want the price feed where the date is on or after 1/1/2022, ordered by the earliest date first. This is what the query would look like:

In [ ]:
#Storing the query to get the Parcl Price Feed for the PHX MSA in 2022
query = """
query PHX_PLPF {
  MSA(where: {MSA_NAME: {_eq: "Phoenix-Mesa-Chandler, AZ"}}) {
    MSA_NAME
      PARCL_ID
      parcl_price_feed(where: {DATE:{_gte: "01/01/2022"}}, order_by: {DATE: asc}) {
        PARCL_PRICE_FEED
        DATE
      }
    }
  }
"""

#Step 3: Call the API and visualize the results

Once you have plugged in your query, the URL and API key you can call the API with Python:

In [ ]:
# Call the API using our query and credentials
response = requests.post(
   url=url,
   json={
     "query": query
   },
   headers={
     "Authorization": f"Bearer {bearer_token}",
     'content-type': "application/json"
   }
)
 
# Parse the JSON response to native Python object
out = response.json()
data = out['data']['MSA'][0]

Finally we can easily flatten the JSON response into a dataframe and visualize it with [Plotly](https://plotly.com/):


In [ ]:
# Flatten the JSON data and store in a dataframe
PHX_PPF_df = pd.json_normalize(data, 'parcl_price_feed', ['MSA_NAME'])
PHX_PPF_df['DATE'] = pd.to_datetime(PHX_PPF_df['DATE'])
 
#Plot the Phoenix Price Feed
PHX_PPF_fig = px.line(PHX_PPF_df, x='DATE', y='PARCL_PRICE_FEED', title='2022 Parcl Price Feed: Phoenix MSA', width=900, height=500,
               labels={
                    "PARCL_PRICE_FEED": "Parcl Price Feed ($)",
                    "DATE": "Date"
                },)

PHX_PPF_fig.update_traces(line_color='#4882db')
PHX_PPF_fig.show()